In [6]:
import pandas as pd
import random
import numpy as np
import math
import copy

In [7]:
hero_list = pd.read_csv('/content/drive/MyDrive/Simulasi Draftpick MCTS/Dataset/hero_list.csv')
hero_list = hero_list['Hero'].tolist()

team_data = pd.read_csv('/content/drive/MyDrive/Simulasi Draftpick MCTS/Dataset/player_data.csv')
available_team = team_data['tim'].unique().tolist()

ban_pick_data = pd.read_csv('/content/drive/MyDrive/Simulasi Draftpick MCTS/Dataset/ban_pick_score.csv')

### Team Selection ###

# print('select your team')
# print('available team: ', available_team)

# while True:
#     team1 = input('team 1:')
#     if team1 in available_team:
#         break
#     print('Invalid team name')

# while True:
#     team2 = input('team 2:')
#     if team2 in available_team:
#         if team2 != team1:
#             break
#         else:
#             print('Team already in team 1, please choose another team')
#     else:
#         print('Invalid team name')

# ### Player Selection ###

# def build_roster(team_name, team_data):

#     roster = []
#     available_players = team_data[team_data['tim'] == team_name]['player'].unique().tolist()
#     lanes = ['Gold laner', 'Mid laner', 'EXP laner', 'Jungler', 'Roamer']

#     print(f'Build {team_name} roster:')
#     print(f'Available players for {team_name}: {available_players}')

#     for lane in lanes:
#         while True:
#             player_select = input(f'Choose the {lane}: ')
#             if player_select in available_players:
#                 if player_select not in roster:
#                     roster.append(player_select)
#                     break
#                 else:
#                     print('Player already in roster')
#             else:
#                 print('Invalid player name')
#     return roster

# team1_roster = build_roster(team1, team_data)
# team2_roster = build_roster(team2, team_data)

team1 = 'Onic'
team2 = 'Alter Ego'
team1_roster = ['CW', 'Sanz', 'Lutpiii', 'Kairi', 'Kiboy']
team2_roster = ['Nino', 'Cr1te', 'Pai', 'Tazz', 'Rasy']

print(team1)
print(team2)
print(team1_roster)
print(team2_roster)

Onic
Alter Ego
['CW', 'Sanz', 'Lutpiii', 'Kairi', 'Kiboy']
['Nino', 'Cr1te', 'Pai', 'Tazz', 'Rasy']


In [8]:
class DraftPick:
    def __init__(self, hero_list, team_data, ban_pick_data, team1, team2):
        self.state = self.get_initial_state(hero_list)
        self.team_data = team_data
        self.ban_pick_data = ban_pick_data
        self.hero_list = hero_list

        self.team1 = team1
        self.team2 = team2

        self.ban_pick_order = [1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2]
        self.action_size = len(hero_list)
        self.camp = self.get_turn_from_state(self.state)

    def get_initial_state(self, hero_list):
        state = {'draft': [],
                 'available_heroes': hero_list.copy(),
                 }
        return state

    def get_valid_moves(self, state):
        #valid moves = available hero to be selected + hero not yet selected + draft is not full
        available_heroes = state['available_heroes']
        already_selected = state['draft']
        valid_moves = [hero for hero in available_heroes if hero not in already_selected]
        return valid_moves

    def get_next_state(self, state, action):
        next_state = copy.deepcopy(state)

        hero = self.get_valid_moves(state)[action]

        next_state['draft'].append(hero)
        next_state['available_heroes'].remove(hero)

        return next_state

    def get_phase_from_state(self, state):
        turn_count = len(state['draft'])
        if turn_count <= 5 or 12 <= turn_count <= 15:
            return 'ban'
        else:
            return 'pick'

    def get_turn_from_state(self, state):
        draft_len = len(state['draft'])
        ban_pick_order = self.ban_pick_order
        turn_index = draft_len % len(ban_pick_order)
        return 'team1' if ban_pick_order[turn_index] == 1 else 'team2'

    # def get_phase(self):
    #     if self.turn_count <= 5 or 11 < self.turn_count <= 15:
    #         phase = 'ban'
    #     else:
    #         phase = 'pick'
    #     return phase

    # def get_turn(self):
    #     ban_pick_order = [1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2]
    #     phase = self.turn_count % len(ban_pick_order)
    #     return 'team1' if ban_pick_order[phase] == 1 else 'team2'

    def get_value_and_terminated(self, state, action):
        value = self.get_score(
            state, ban_pick_data, self.team_data, team1, team2, team1_roster, team2_roster
        )
        if len(state['draft']) >= 20: #Indicating that it reach the end of simulation (is_terminal)
            return value, True
        else:
            return value, False


    def get_score(self, state, ban_pick_data, team_data, team1, team2, team1_roster, team2_roster):

        draft = state['draft']
        ban_pick_order = self.ban_pick_order

        team1_pick = []
        team2_pick = []
        team1_ban = []
        team2_ban = []

        for x in range(min(len(draft), len(ban_pick_order))):
            if x <= 5 or 12 <= x <= 15:
                if ban_pick_order[x] == 1:
                    team1_ban.append(draft[x])
                else:
                    team2_ban.append(draft[x])
            else:
                if ban_pick_order[x] == 1:
                    team1_pick.append(draft[x])
                else:
                    team2_pick.append(draft[x])

        team1_ban_score = self.calculate_ban_score(team_data, team2, team2_roster, team1_ban)
        team2_ban_score = self.calculate_ban_score(team_data, team1, team1_roster, team2_ban)
        team1_pick_score = self.calculate_pick_score(team_data, team1, team1_roster, team1_pick)
        team2_pick_score = self.calculate_pick_score(team_data, team2, team2_roster, team2_pick)

        team1_score_list = {**team1_ban_score, **team1_pick_score}
        team2_score_list = {**team2_ban_score, **team2_pick_score}

        all_team_score = {'team1': (sum(team1_score_list.values()) if team1_score_list else 0),
                          'team2': (sum(team2_score_list.values()) if team2_score_list else 0)}

        '''''''''''''''''' "if need all hero score in a list" ''''''''''''''''''

        # all_score = {**team1_score, **team2_score}
        # score_list = [all_score[i] for i in draft]

        return all_team_score

    def calculate_ban_score(self, team_data, team_name, roster, ban_list):
        ban_scores = {}
        for hero in ban_list:
            player_scores = {}
            for player in roster:
                if hero in team_data[(team_data['tim'] == team_name) & (team_data['player'] == player)]['hero'].unique().tolist():
                    normalized_value = team_data[(team_data['tim'] == team_name) & (team_data['player'] == player) & (team_data['hero'] == hero)]['normalized'].item()
                    player_scores[player] = normalized_value
            if player_scores:  # Check if any players have scores for this hero
                ban_scores[hero] = max(player_scores.values())  # Append the maximum score to the list
            else:
                ban_scores[hero] = 0  # Append 0 if no score found for the hero
        return ban_scores

    def calculate_pick_score(self, team_data, team_name, roster, pick_list):
        if not pick_list:
            return {hero: 0 for hero in self.hero_list}
        pick_scores = {}
        player_with_hero = {}
        unique_player = {} #player who only play one hero

        for hero in pick_list:
            player_with_hero[hero] = {}
            for player in roster:
                if hero in team_data[(team_data['tim'] == team_name) & (team_data['player'] == player)]['hero'].unique().tolist():
                    normalized_value = team_data[(team_data['tim'] == team_name) & (team_data['player'] == player) & (team_data['hero'] == hero)]['normalized'].item()
                    player_with_hero[hero][player] = normalized_value

        for hero, player_score in player_with_hero.items():
            if len(player_score) == 1:
                unique_player.update(player_score)

        for hero, player_score in player_with_hero.items():
            if len(player_score) > 1:
                for player in list(player_score.keys()):
                    if player in unique_player:
                        del player_score[player]

        for player in unique_player:
            max_value = 0
            max_hero = None
            for hero, player_score in player_with_hero.items():
                if player in player_score and player_score[player] > max_value:
                    max_value = player_score[player]
                    max_hero = hero
            for hero, player_score in player_with_hero.items():
                if player in player_score and hero != max_hero:
                    del player_score[player]

        for hero, player_score in player_with_hero.items():
            if len(player_score) > 1:
                max_value = max(player_score.values())
                for player in list(player_score.keys()):
                    if player_score[player] < max_value:
                        del player_score[player]

        for hero, player_score in player_with_hero.items():
            if player_score:
                pick_scores[hero] = list(player_score.values())[0]
            else:
                pick_scores[hero] = 0  # Handle empty player_score

        return pick_scores

    def get_meta_score(self, ban_pick_data, draft_result, camp, phase):
        meta_score = 0

        if phase == 'ban':
            num_heroes_to_check = 3
        elif phase == 'pick' and camp == 1:
            num_heroes_to_check = 1
        else:
            num_heroes_to_check = 2

        if camp == 'team1':
            start_from = 0
        elif camp == 'team2':
            start_from = 1

        score_column = 'ban_score' if phase == 'ban' else 'pick_score'

        for hero in draft_result[:num_heroes_to_check]:
            if hero in ban_pick_data['hero'].unique().tolist():
                score = ban_pick_data[ban_pick_data['hero'] == hero][score_column].iloc[0]
            else:
                score = -1
            meta_score += score

        for hero in draft_result[start_from:3]:
            if hero in ban_pick_data['hero'].unique().tolist():
                score = ban_pick_data[ban_pick_data['hero'] == hero][score_column].iloc[0] * 0.5
            else:
                score = -1
            meta_score += score

        for hero in draft_result[3:]:
            if hero not in ban_pick_data['hero'].unique().tolist():
                score = -1
            meta_score += score

        return meta_score

In [9]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken

        self.children = []

        self.visit_count = 0
        self.value_sum = 0

    def is_fully_expanded(self):
        return len(self.children) > 0 and len(self.children) == len(self.game.get_valid_moves(self.state))

    def select(self):
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb

        return best_child

    def get_ucb(self, child):
        q_value = 1 - (child.value_sum / child.visit_count)
        priority_bonus = 0
        if child.action_taken is not None:
            hero = self.game.get_valid_moves(self.state)[child.action_taken]
            hero_data = self.game.ban_pick_data[self.game.ban_pick_data['hero'] == hero]
            if not hero_data.empty:
                ban_score = hero_data['ban_score'].iloc[0]
                pick_score = hero_data['pick_score'].iloc[0]
                priority_score = ban_score + pick_score
                priority_bonus = priority_score / 2

            current_draft_index = len(self.game.state['draft'])
            child_draft_index = len(child.state['draft'])
            if child_draft_index == current_draft_index + 1:
                if (child_draft_index <= 5 or 12 <= child_draft_index <= 15) and self.game.calculate_ban_score(team_data, team2, team2_roster, [hero])[hero] == 0:
                    q_value -= 2

                elif (6 <= child_draft_index <= 11 or child_draft_index >= 16) and self.game.calculate_pick_score(team_data, team1, team1_roster, [hero])[hero] == 0: #Replace the condition
                    q_value -= 2

        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count) + priority_bonus

    def expand(self):
        valid_moves = self.game.get_valid_moves(self.state)
        action = random.choice(range(len(valid_moves)))

        child_state = self.state.copy()
        child_state = self.game.get_next_state(child_state, action)  # Call on self.game

        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)
        return child

    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        rollout_state = self.state.copy()

        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = random.choice(range(len(valid_moves))) #Both Player Use Random

            rollout_state = self.game.get_next_state(rollout_state, action)
            # print("draft_state:", rollout_state['draft']) # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< debugging, delete later
            # print("valid_moves:", valid_moves) # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< debugging, delete later

            current_draft_index = len(self.game.state['draft'])
            rollout_draft_index = len(rollout_state['draft'])

            if rollout_draft_index == current_draft_index + 1:
                punishment_value = 100
                hero = self.game.get_valid_moves(rollout_state)[action] # Get the chosen hero
                if (rollout_draft_index <= 5 or 12 <= rollout_draft_index <= 15) and self.game.calculate_ban_score(self.game.team_data, self.game.team2, self.game.team2_roster, [hero])[hero] == 0:
                    value -= punishment_value

                elif (6 <= rollout_draft_index <= 11 or rollout_draft_index >= 16) and self.game.calculate_pick_score(self.game.team_data, self.game.team1, self.game.team1_roster, [hero])[hero] == 0: #Replace the condition
                    value -= punishment_value

            all_team_score, is_terminal = self.game.get_value_and_terminated(rollout_state, action)
            rollout_player = self.game.get_turn_from_state(rollout_state)

            if is_terminal:
                camp = self.game.get_turn_from_state(rollout_state)
                value = all_team_score['team1'] if camp == 'team1' else all_team_score['team2'] # Directly access team1_score
                break

        return value

    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        if self.parent is not None:
            if self.parent.game.get_turn_from_state(self.parent.state) == 'team2':
                value = -value
            self.parent.backpropagate(value)

class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args

    def search(self, state):
        root = Node(self.game, self.args, state)

        for search in range(self.args['num_searches']):
            node = root

            while node.is_fully_expanded():
                node = node.select()

            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = value['team1'] #depending on which side is the MCTS

            if not is_terminal:
                node = node.expand()
                value = node.simulate()

            node.backpropagate(value)

        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs


In [10]:
### Draft Pick ###

DraftPick = DraftPick(hero_list, team_data, ban_pick_data, team1, team2)

args = {
    'num_searches': 1000,
    'C': 0.7,  # Exploration constant (higher number mean more explore, lower number mean more exploitation. Midle number = 1.4)
}
mcts = MCTS(DraftPick, args)

while len(DraftPick.state['draft']) < 20:
    state = DraftPick.state
    print(state)

    team_turn = DraftPick.get_turn_from_state(state)
    phase = DraftPick.get_phase_from_state(state)
    valid_moves = DraftPick.get_valid_moves(state)

    print(f"{team_turn}'s turn to {phase}:")
    # print('Available heroes:', valid_moves)

    if team_turn == 'team1':  # Team 1 use MCTS
        action_probs = mcts.search(state)
        print(action_probs)  # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< debugging, delete later
        action = np.argmax(action_probs)
        print(action) # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< debugging, delete later
        chosen_hero = valid_moves[action]
        print(f'Team 1 {phase}s: {chosen_hero}')
        DraftPick.state = DraftPick.get_next_state(state, action)
        # print(DraftPick.state)
    else:  # Random choice for team2
        chosen_hero = random.choice(valid_moves)
        print(f'Team 2 {phase}s: {chosen_hero}')
        action = valid_moves.index(chosen_hero)
        DraftPick.state = DraftPick.get_next_state(state, action)

print('Draft completed!')
print(DraftPick.state)

{'draft': [], 'available_heroes': ['Miya', 'Balmond', 'Saber', 'Alice', 'Nana', 'Tigreal', 'Alucard', 'Karina', 'Akai', 'Franco', 'Bane', 'Bruno', 'Clint', 'Rafaela', 'Eudora', 'Zilong', 'Fanny', 'Layla', 'Minotaur', 'Lolita', 'Hayabusa', 'Freya', 'Gord', 'Natalia', 'Kagura', 'Chou', 'Sun', 'Alpha', 'Ruby', 'Yi Sun-shin', 'Moskov', 'Johnson', 'Cyclops', 'Estes', 'Hilda', 'Aurora', 'Lapu-Lapu', 'Vexana', 'Roger', 'Karrie', 'Gatotkaca', 'Harley', 'Irithel', 'Grock', 'Argus', 'Odette', 'Lancelot', 'Diggie', 'Hylos', 'Zhask', 'Helcurt', 'Pharsa', 'Lesley', 'Jawhead', 'Angela', 'Gusion', 'Valir', 'Martis', 'Uranus', 'Hanabi', "Chang'e", 'Kaja', 'Selena', 'Aldous', 'Claude', 'Vale', 'Leomord', 'Lunox', 'Hanzo', 'Belerick', 'Kimmy', 'Thamuz', 'Harith', 'Minsitthar', 'Kadita', 'Faramis', 'Badang', 'Kufra', 'Granger', 'Guinevere', 'Esmeralda', 'Terizla', 'X-Borg', 'Ling', 'Dyrroth', 'Lylia', 'Baxia', 'Masha', 'Wanwan', 'Silvana', 'Carmilla', 'Cecilion', 'Atlas', 'Popol and Kupa', 'Yu Zhong', 'L

In [11]:
draft = DraftPick.state['draft']

print(len(draft))

team1_ban = [draft[i] for i in [0, 2, 4, 13, 14]]
team1_pick = [draft[i] for i in [6, 9, 10, 17, 18]]
team2_ban = [draft[i] for i in [1, 3, 5, 12, 15]]
team2_pick = [draft[i] for i in [7, 8, 11, 16, 19]]

print(f'{team1} ban:{team1_ban}')
print(f'{team1} pick:{team1_pick}')
print(f'{team2} ban:{team2_ban}')
print(f'{team2} pick:{team2_pick}')

20
Onic ban:['Fredrinn', 'Miya', 'Lylia', 'Bruno', 'Alice']
Onic pick:['Ruby', 'Mathilda', 'Yu Zhong', 'Arlott', 'Minotaur']
Alter Ego ban:['Benedetta', 'Barats', 'Beatrix', 'Jawhead', 'Granger']
Alter Ego pick:['Ling', 'Hylos', 'Lunox', 'Baxia', 'Carmilla']


In [12]:
state = DraftPick.state
final_score = DraftPick.get_score(state, ban_pick_data, team_data, team1, team2, team1_roster, team2_roster)
# team2_score = DraftPick.get_score(state, ban_pick_data, team_data, team1, team2, team1_roster, team2_roster)

print(final_score)
# print(team1_pick)
# print(f'{team1} draft score: {team1_score}')
# print(team2_pick)
# print(f'{team2} draft score: {team2_score}')

{'team1': 6.889751081807178, 'team2': 5.848700150866472}
